# History Key Default Value Fix Test

Test the fixed history_key functionality to ensure first call doesn't fail due to missing key

In [1]:
from llm_wrapper import Agents
from llm_models import llm_4o_mini
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


test 0 wo chat history

In [ ]:
# this prompt not requires a custom history key
test_prompt = """
Generate 10 points intotal. content about {topic}. 
when generate 4 points then output "<continue>".
the number should be continuesly sequence
"""
test_chain = Agents.chain_create(text_prompt_template=test_prompt,model=llm_4o_mini,)

try:
    result = Agents.continue_chain_batch_generator(
        chain=test_chain,
        dic={"topic": "artificial intelligence ethics"},  # Note: no my_custom_history
        max_continue=2,
        trigger_type="str",  #
        continue_trigger="<continue>",
    )
    print("✅ Success! No error")
except Exception as e:
    print("❌ Error:", e)

In [11]:
result

"1. **Transparency**: One of the key ethical considerations in artificial intelligence is the need for transparency in AI systems. Users and stakeholders should understand how AI algorithms make decisions, which can help build trust and accountability.\n\n2. **Bias and Fairness**: AI systems can inadvertently perpetuate or amplify biases present in their training data. It is crucial to ensure that AI applications are designed to be fair and equitable, minimizing discrimination against any group based on race, gender, or socioeconomic status.\n\n3. **Privacy**: The use of AI often involves the collection and analysis of vast amounts of personal data. Ethical AI development must prioritize user privacy, ensuring that data is collected, stored, and used in ways that respect individuals' rights and consent.\n\n4. **Accountability**: As AI systems become more autonomous, determining accountability for their actions becomes complex. Establishing clear lines of responsibility for AI decisions

## Test 1: Verify First Call Doesn't Error

In [2]:
# This prompt requires a custom history key
test_prompt = """
Generate content about {topic}. 10 points intotal.

History log: {my_custom_history}

If there's history, continue based on the history.
Generate 4 points then output "continue.......".
"""

test_chain = Agents.chain_create(
    model=llm_4o_mini,
    text_prompt_template=test_prompt
)

print("=== Test First Call No Error ===")

# Here dic doesn't contain my_custom_history, but should not error
try:
    result = Agents.continue_chain_batch_generator(
        chain=test_chain,
        dic={"topic": "artificial intelligence ethics"},  # Note: no my_custom_history
        max_continue=2,
        trigger_type="str",  # 
        continue_trigger="continue",
        history_key="my_custom_history"  # But specified this key
    )
    print("✅ Success! No error")
    print(f"Result length: {len(result)}")
    print("First 300 characters:")
    print(result[:300])
except Exception as e:
    print(f"❌ Failed: {e}")

=== Test First Call No Error ===
✅ Success! No error
Result length: 3398
First 300 characters:
1. **Definition of AI Ethics**: AI ethics refers to the moral principles and guidelines that govern the development and deployment of artificial intelligence technologies. It encompasses issues such as fairness, accountability, transparency, and the impact of AI on society.

2. **Bias and Fairness**


In [4]:
result

"1. **Definition of AI Ethics**: AI ethics refers to the moral principles and guidelines that govern the development and deployment of artificial intelligence technologies. It encompasses issues such as fairness, accountability, transparency, and the impact of AI on society.\n\n2. **Bias and Fairness**: One of the primary concerns in AI ethics is the potential for bias in AI algorithms. If the data used to train AI systems is biased, the outcomes can perpetuate discrimination against certain groups. Ensuring fairness involves actively identifying and mitigating these biases.\n\n3. **Accountability**: As AI systems become more autonomous, determining accountability for their actions becomes complex. Questions arise about who is responsible when an AI system causes harm or makes a mistake—whether it be the developers, users, or the AI itself.\n\n4. **Privacy and Surveillance**: The use of AI in surveillance technologies raises significant ethical concerns regarding privacy. The ability o

test json

In [5]:
test_prompt = """
  generate content about {topic}. 10 points intotal.
  History log: {my_custom_history}
  If there's history, continue based on the history.

  IMPORTANT: Output ONLY valid JSON without any markdown formatting or code 
  blocks.

  Format:
  [
      {{"number": 1, "content": "point 1 xxxx"}},
      {{"number": 2, "content": "point 2 xxxx"}},
      ...
      {{"continue": "continue"}}
  ]

  Generate 4 points then add the continue marker in the JSON array.
  """

test_chain = Agents.chain_create(model=llm_4o_mini, text_prompt_template=test_prompt, output_parser=JsonOutputParser())

try:
    result = Agents.continue_chain_batch_generator(
        chain=test_chain,
        dic={"topic": "artificial intelligence ethics"},  # Note: no my_custom_history
        max_continue=2,
        trigger_type="json",
        continue_trigger={"continue": "continue"},  # New simple JSON format
        history_key="my_custom_history"  # But specified this key
    )
    print("✅ Success! No error")
    print(f"Result length: {len(result)}")
    print("First 300 characters:")
    print(result[:300])
except Exception as e:
    print(f"❌ Failed: {e}")

✅ Success! No error
Result length: 1774
First 300 characters:
[{'number': 1, 'content': 'Artificial intelligence ethics examines the moral implications and responsibilities associated with the development and deployment of AI technologies.'}, {'number': 2, 'content': 'Key ethical concerns include bias in AI algorithms, which can lead to unfair treatment of ind


In [6]:
result

"[{'number': 1, 'content': 'Artificial intelligence ethics examines the moral implications and responsibilities associated with the development and deployment of AI technologies.'}, {'number': 2, 'content': 'Key ethical concerns include bias in AI algorithms, which can lead to unfair treatment of individuals based on race, gender, or socioeconomic status.'}, {'number': 3, 'content': 'Transparency in AI decision-making processes is crucial to ensure accountability and trust among users and stakeholders.'}, {'number': 4, 'content': 'The potential for job displacement due to automation raises ethical questions about the responsibility of companies and governments to support affected workers.'}, ]\n[{'number': 5, 'content': 'The use of AI in surveillance raises ethical concerns regarding privacy and the potential for abuse of power by authorities.'}, {'number': 6, 'content': 'Informed consent is essential when deploying AI systems, particularly in healthcare, where patients should understa